# Training UniVL FROM SCRATCH trên MSRVTT

## 🚀 Mục đích

Notebook này hướng dẫn training UniVL **KHÔNG sử dụng pretrained weights** (`univl.pretrained.bin`).

### So sánh với training thông thường:

| Aspect | Training Thông Thường | Training FROM SCRATCH |
|--------|----------------------|----------------------|
| **Pretrained** | ✅ Dùng `univl.pretrained.bin` | ❌ KHÔNG dùng pretrained |
| **BERT** | ✅ Pretrained | ✅ Pretrained (vẫn dùng) |
| **Visual Encoder** | ✅ Pretrained (HowTo100M) | ❌ Random initialization |
| **Cross Encoder** | ✅ Pretrained | ❌ Random initialization |
| **Decoder** | ✅ Pretrained | ❌ Random initialization |
| **Epochs** | 5 | 10-15 (cần nhiều hơn) |
| **Learning Rate** | 3e-5 | 5e-5 (cao hơn) |
| **Kết quả** | BLEU-4: ~0.42 | BLEU-4: ~0.30-0.35 (thường thấp hơn) |
| **Training Time** | 3-6 giờ | 6-12 giờ |

### Khi nào nên training from scratch?

✅ **Nên dùng khi:**
- Dataset của bạn RẤT khác biệt so với HowTo100M
- Muốn thử nghiệm architecture mới
- Research về transfer learning
- Có GPU/TPU resources mạnh và thời gian

❌ **KHÔNG nên dùng khi:**
- Chỉ muốn kết quả tốt nhanh → Dùng pretrained
- Dataset nhỏ (< 10k videos) → Dùng pretrained
- GPU/compute resources hạn chế → Dùng pretrained

---

## Yêu cầu môi trường

- **GPU**: T4 x2 hoặc P100 (Kaggle)
- **RAM**: ≥ 16GB
- **Disk**: ≥ 20GB free space
- **Time**: 6-12 giờ training

---
## Bước 0: Clone Source Code và Kiểm tra Môi trường

### 0.1 Clone UniVL repository

In [ ]:
# Clone UniVL repository
!git clone https://github.com/microsoft/UniVL.git
%cd UniVL

print("✓ Repository cloned!")
!ls -la

In [ ]:
# Kiểm tra môi trường
import torch
import sys

print(f"Python version: {sys.version}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
print(f"Number of GPUs: {torch.cuda.device_count()}")

if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

In [ ]:
# Cài đặt dependencies
!pip install tqdm pandas
!pip install pycocoevalcap

print("✓ Dependencies installed!")

---
## Bước 1: Tải BERT Pretrained Model

**Lưu ý quan trọng:** Dù training từ đầu (from scratch), chúng ta vẫn cần **BERT pretrained model** cho Text Encoder. Phần "from scratch" ở đây nghĩa là:
- ❌ KHÔNG dùng UniVL pretrained weights (univl.pretrained.bin) - đã được pretrain trên HowTo100M
- ✅ VẪN dùng BERT base pretrained (bert-base-uncased) - chỉ là BERT thông thường
- ✅ Visual Encoder, Cross Encoder, Decoder sẽ được khởi tạo ngẫu nhiên

Tải về `bert-base-uncased`:

In [ ]:
# Tạo thư mục và tải BERT base model
!mkdir -p bert-base-uncased
!wget -P bert-base-uncased https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt
!wget -P bert-base-uncased https://huggingface.co/bert-base-uncased/resolve/main/config.json
!wget -P bert-base-uncased https://huggingface.co/bert-base-uncased/resolve/main/pytorch_model.bin

print("✓ BERT model downloaded!")

---
## Bước 2: Tải Dataset MSRVTT

**Lưu ý:** Chúng ta KHÔNG tải UniVL pretrained weights (univl.pretrained.bin) vì đang training from scratch.

In [ ]:
# Tải MSRVTT dataset
!mkdir -p data/msrvtt

# Download video features (S3D features extracted)
!wget -P data/msrvtt https://www.rocq.inria.fr/cluster-willow/amiech/MSRVTT.zip
!unzip -q data/msrvtt/MSRVTT.zip -d data/msrvtt/

# Download captions
!wget -P data/msrvtt https://github.com/ArrowLuo/CLIP4Clip/releases/download/v0.0/msrvtt_data.zip
!unzip -q data/msrvtt/msrvtt_data.zip -d data/msrvtt/

print("✓ MSRVTT dataset downloaded!")
!ls -lh data/msrvtt/

---
## Bước 3: Chuẩn bị Training Command (From Scratch)

### Các thay đổi quan trọng khi training from scratch:

| Tham số | With Pretrained | From Scratch | Lý do |
|---------|----------------|--------------|-------|
| `--init_model` | ✅ Có (univl.pretrained.bin) | ❌ BỎ ĐI | Không dùng pretrained weights |
| `--epochs` | 5 | **10-15** | Cần nhiều epoch hơn để học từ đầu |
| `--lr` | 3e-5 | **5e-5** | Learning rate cao hơn cho random init |
| `--batch_size` | 128 | **64-96** | Có thể giảm nếu thiếu RAM |
| `--bert_model` | ✅ Vẫn cần | ✅ Vẫn cần | BERT base vẫn được pretrain |

### Training strategy:
1. **Visual Encoder**: Khởi tạo ngẫu nhiên (không có HowTo100M pretrained)
2. **Text Encoder**: BERT base pretrained (bert-base-uncased)
3. **Cross Encoder & Decoder**: Khởi tạo ngẫu nhiên

In [ ]:
# Tạo training command (from scratch - NO pretrained weights)
DATA_PATH = "data/msrvtt"
OUTPUT_DIR = "outputs/msrvtt_caption_fromscratch"
BERT_PATH = "bert-base-uncased"

# Số GPU và batch size
NUM_GPUS = torch.cuda.device_count()
BATCH_SIZE_PER_GPU = 32  # Điều chỉnh theo VRAM của GPU
TOTAL_BATCH_SIZE = BATCH_SIZE_PER_GPU * NUM_GPUS

print(f"Number of GPUs: {NUM_GPUS}")
print(f"Batch size per GPU: {BATCH_SIZE_PER_GPU}")
print(f"Total batch size: {TOTAL_BATCH_SIZE}")
print()

# ⚠️ LƯU Ý: KHÔNG có --init_model (đây là điểm khác biệt chính!)
training_command = f"""
torchrun --nproc_per_node={NUM_GPUS} \\
  --master_port=2502 \\
  main_task_caption.py \\
  --do_train \\
  --num_thread_reader=4 \\
  --epochs=12 \\
  --batch_size={BATCH_SIZE_PER_GPU} \\
  --n_display=50 \\
  --train_csv {DATA_PATH}/MSRVTT_train.9k.csv \\
  --val_csv {DATA_PATH}/MSRVTT_JSFUSION_test.csv \\
  --data_path {DATA_PATH}/MSRVTT_data.json \\
  --features_path {DATA_PATH}/MSRVTT_Videos \\
  --output_dir {OUTPUT_DIR} \\
  --bert_model {BERT_PATH} \\
  --visual_model modules/visual-base/visual_config.json \\
  --cross_model modules/cross-base/cross_config.json \\
  --decoder_model modules/decoder-base/decoder_config.json \\
  --max_words 32 \\
  --max_frames 12 \\
  --video_framerate 1 \\
  --lr 5e-5 \\
  --gradient_accumulation_steps 1 \\
  --coef_lr 1e-3 \\
  --warmup_proportion 0.1 \\
  --n_gpu {NUM_GPUS}
""".strip()

print("=" * 80)
print("TRAINING COMMAND (FROM SCRATCH):")
print("=" * 80)
print(training_command)
print("=" * 80)
print()
print("⚠️  Lưu ý: KHÔNG có --init_model, model sẽ training từ đầu!")
print("⏱  Thời gian dự kiến: 2-3 ngày (phụ thuộc GPU)")
print("📊 Validation loss sẽ được hiển thị sau mỗi epoch")

In [ ]:
# Tạo output directory
!mkdir -p {OUTPUT_DIR}

# Chạy training
!{training_command}

---
## Bước 4: Monitoring Training Progress

Sau khi chạy training, bạn có thể theo dõi:
- **Training loss**: Xuất hiện mỗi 50 steps (`n_display=50`)
- **Validation metrics**: Sau mỗi epoch, hiển thị:
  - BLEU-1, BLEU-2, BLEU-3, BLEU-4
  - METEOR, ROUGE_L, CIDEr
  - **Validation Loss** (đã được thêm vào code)

### Các file output quan trọng:
- `{OUTPUT_DIR}/pytorch_model.bin.{epoch}` - Model checkpoint mỗi epoch
- `{OUTPUT_DIR}/log.txt` - Training log

In [ ]:
# Xem training log trong thời gian thực
!tail -f {OUTPUT_DIR}/log.txt

---
## Bước 5: Evaluation trên Test Set

Sau khi training xong, đánh giá model tốt nhất trên test set:

In [ ]:
# Tìm epoch tốt nhất (dựa vào validation BLEU-4)
import os
import glob

checkpoint_files = glob.glob(f"{OUTPUT_DIR}/pytorch_model.bin.*")
if checkpoint_files:
    print("Available checkpoints:")
    for ckpt in sorted(checkpoint_files):
        epoch = ckpt.split('.')[-1]
        print(f"  - Epoch {epoch}: {ckpt}")
        
    # Thường chọn epoch cuối hoặc epoch có BLEU-4 cao nhất (xem trong log.txt)
    best_epoch = len(checkpoint_files)  # Mặc định chọn epoch cuối
    print(f"\n📌 Using checkpoint from epoch {best_epoch}")
else:
    print("❌ No checkpoints found!")

In [ ]:
# Chạy evaluation trên test set
BEST_MODEL = f"{OUTPUT_DIR}/pytorch_model.bin.{best_epoch}"

eval_command = f"""
torchrun --nproc_per_node={NUM_GPUS} \\
  --master_port=2502 \\
  main_task_caption.py \\
  --do_eval \\
  --num_thread_reader=4 \\
  --batch_size={BATCH_SIZE_PER_GPU} \\
  --val_csv {DATA_PATH}/MSRVTT_JSFUSION_test.csv \\
  --data_path {DATA_PATH}/MSRVTT_data.json \\
  --features_path {DATA_PATH}/MSRVTT_Videos \\
  --output_dir {OUTPUT_DIR} \\
  --bert_model {BERT_PATH} \\
  --visual_model modules/visual-base/visual_config.json \\
  --cross_model modules/cross-base/cross_config.json \\
  --decoder_model modules/decoder-base/decoder_config.json \\
  --init_model {BEST_MODEL} \\
  --max_words 32 \\
  --max_frames 12 \\
  --video_framerate 1 \\
  --n_gpu {NUM_GPUS}
""".strip()

print("EVALUATION COMMAND:")
print("=" * 80)
print(eval_command)
print("=" * 80)

!{eval_command}

---
## Bước 6 (Optional): So sánh với Pretrained Model

Nếu muốn so sánh kết quả training from scratch với pretrained model, có thể tải univl.pretrained.bin và chạy training với cùng hyperparameters:

In [ ]:
# (Optional) Download pretrained UniVL để so sánh
!wget https://github.com/microsoft/UniVL/releases/download/v0/univl.pretrained.bin

print("✓ Pretrained model downloaded for comparison!")
print("\nĐể train với pretrained, thêm flag:")
print("  --init_model univl.pretrained.bin")
print("\nVà giảm epochs xuống 5, lr xuống 3e-5")

---
## 📊 Kết quả dự kiến

### Training from Scratch (dự kiến sau 12 epochs):
- **BLEU-4**: ~35-38 (thấp hơn pretrained ~2-5 điểm)
- **CIDEr**: ~40-45
- **Training time**: 2-3 ngày trên 4x V100

### With Pretrained (5 epochs):
- **BLEU-4**: ~40-42
- **CIDEr**: ~47-50
- **Training time**: 12-18 giờ trên 4x V100

### Lợi ích của training from scratch:
- ✅ Không phụ thuộc vào HowTo100M pretrained
- ✅ Có thể customize architecture tự do
- ✅ Hiểu rõ model behavior từ đầu
- ❌ Kết quả thấp hơn ~5-10%
- ❌ Training lâu hơn 3-5 lần

---

## 🎯 Tổng kết

Notebook này đã hướng dẫn bạn:
1. ✅ Setup môi trường và dependencies
2. ✅ Tải BERT base model (vẫn cần cho Text Encoder)
3. ✅ Tải MSRVTT dataset (không cần UniVL pretrained)
4. ✅ Training command KHÔNG có --init_model
5. ✅ Monitoring và evaluation
6. ✅ So sánh với pretrained model (optional)

**Chúc bạn training thành công! 🚀**